<img src="https://i.imgur.com/FoKB5Z5.png" align="left" width="300" height="250" title="source: imgur.com" /></a>

## Program Code: J620-002-4:2020 

## Program Name: FRONT-END SOFTWARE DEVELOPMENT

## Title : P57 - Neural Networking using Keras

#### Name: 

#### IC Number:

#### Date :

#### Introduction : 



#### Conclusion :






# Multi-Class Classification Tutorial with the Keras Deep Learning Library

Adapted from: https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/

## Problem Description
In this tutorial, we will use the standard machine learning problem called the iris flowers dataset.

This dataset is well studied and is a good problem for practicing on neural networks because all of the 4 input variables are numeric and have the same scale in centimeters. Each instance describes the properties of an observed flower measurements and the output variable is specific iris species.

This is a multi-class classification problem, meaning that there are more than two classes to be predicted, in fact there are three flower species. This is an important type of problem on which to practice with neural networks because the three class values require specialized handling.

The iris flower dataset is a well-studied problem and a such we can expect to achieve a model accuracy in the range of 95% to 97%. This provides a good target to aim for when developing our models.

You can download the iris flowers dataset from the [UCI Machine Learning repository](http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data) and place it in your current working directory with the filename “iris.csv“. You can also use the iris dataset from sklearn datasets.

## Import Classes and Functions
We can begin by importing all of the classes and functions we will need in this tutorial.

This includes both the functionality we require from Keras, but also data loading from pandas as well as data preparation and model evaluation from scikit-learn.

In [5]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris

## Load The Dataset
The dataset can be loaded directly. Because the output variable contains strings, it is easiest to load the data using pandas. We can then split the attributes (columns) into input variables (X) and output variables (Y).

In [6]:
# load dataset
iris = load_iris()

dataframe = pd.DataFrame(data= np.c_[iris['data'], iris['target']], columns= iris['feature_names'] + ['target'])
# dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

## Encode The Output Variable
The output variable contains three different string values.

When modeling multi-class classification problems using neural networks, it is good practice to reshape the output attribute from a vector that contains values for each class value to be a matrix with a boolean for each class value and whether or not a given instance has that class value or not.

This is called [one hot encoding](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/) or creating dummy variables from a categorical variable.

For example, in this problem three class values are Iris-setosa, Iris-versicolor and Iris-virginica. If we had the observations:

In [7]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

## Define The Neural Network Model
If you are new to Keras or deep learning, see this [helpful Keras tutorial](https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/).

The Keras library provides wrapper classes to allow you to use neural network models developed with Keras in scikit-learn.

There is a KerasClassifier class in Keras that can be used as an Estimator in scikit-learn, the base type of model in the library. The KerasClassifier takes the name of a function as an argument. This function must return the constructed neural network model, ready for training.

Below is a function that will create a baseline neural network for the iris classification problem. It creates a simple fully connected network with one hidden layer that contains 8 neurons.

The hidden layer uses a rectifier activation function which is a good practice. Because we used a one-hot encoding for our iris dataset, the output layer must create 3 output values, one for each class. The output value with the largest value will be taken as the class predicted by the model.

The network topology of this simple one-layer neural network can be summarized as:

    4 inputs -> [8 hidden nodes] -> 3 outputs
   
Note that we use a “softmax” activation function in the output layer. This is to ensure the output values are in the range of 0 and 1 and may be used as predicted probabilities.

Finally, the network uses the efficient Adam gradient descent optimization algorithm with a logarithmic loss function, which is called “categorical_crossentropy” in Keras.

In [21]:
# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=4, activation='relu'))
	model.add(Dense(4, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

We can now create our KerasClassifier for use in scikit-learn.

We can also pass arguments in the construction of the KerasClassifier class that will be passed on to the fit() function internally used to train the neural network. Here, we pass the number of epochs as 200 and batch size as 5 to use when training the model. Debugging is also turned off when training by setting verbose to 0.

In [22]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=2)


C:\Users\Asus\AppData\Local\Temp\ipykernel_5956\1803887780.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=2)


## Evaluate The Model with k-Fold Cross Validation
We can now evaluate the neural network model on our training data.

The scikit-learn has excellent capability to evaluate models using a suite of techniques. The gold standard for evaluating machine learning models is k-fold cross validation.

First we can define the model evaluation procedure. Here, we set the number of folds to be 10 (an excellent default) and to shuffle the data before partitioning it. 

In [23]:
kfold = KFold(n_splits=10, shuffle=True)


Now we can evaluate our model (estimator) on our dataset (X and dummy_y) using a 10-fold cross-validation procedure (kfold).

Evaluating the model only takes approximately 10 seconds and returns an object that describes the evaluation of the 10 constructed models for each of the splits of the dataset. *Note*: This step/process may take a while.

In [24]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/200
27/27 - 0s - loss: 1.1302 - accuracy: 0.3259 - 452ms/epoch - 17ms/step
Epoch 2/200
27/27 - 0s - loss: 1.0932 - accuracy: 0.3259 - 37ms/epoch - 1ms/step
Epoch 3/200
27/27 - 0s - loss: 1.0725 - accuracy: 0.3259 - 35ms/epoch - 1ms/step
Epoch 4/200
27/27 - 0s - loss: 1.0550 - accuracy: 0.3259 - 38ms/epoch - 1ms/step
Epoch 5/200
27/27 - 0s - loss: 1.0373 - accuracy: 0.3259 - 34ms/epoch - 1ms/step
Epoch 6/200
27/27 - 0s - loss: 1.0196 - accuracy: 0.3259 - 37ms/epoch - 1ms/step
Epoch 7/200
27/27 - 0s - loss: 1.0008 - accuracy: 0.2148 - 34ms/epoch - 1ms/step
Epoch 8/200
27/27 - 0s - loss: 0.9741 - accuracy: 0.4667 - 32ms/epoch - 1ms/step
Epoch 9/200
27/27 - 0s - loss: 0.9438 - accuracy: 0.6444 - 34ms/epoch - 1ms/step
Epoch 10/200
27/27 - 0s - loss: 0.9121 - accuracy: 0.6741 - 34ms/epoch - 1ms/step
Epoch 11/200
27/27 - 0s - loss: 0.8852 - accuracy: 0.6667 - 32ms/epoch - 1ms/step
Epoch 12/200
27/27 - 0s - loss: 0.8086 - accuracy: 0.6741 - 35ms/epoch - 1ms/step
Epoch 13/200
27/27 - 0s

Epoch 101/200
27/27 - 0s - loss: 0.1400 - accuracy: 0.9704 - 33ms/epoch - 1ms/step
Epoch 102/200
27/27 - 0s - loss: 0.1333 - accuracy: 0.9852 - 33ms/epoch - 1ms/step
Epoch 103/200
27/27 - 0s - loss: 0.1303 - accuracy: 0.9778 - 35ms/epoch - 1ms/step
Epoch 104/200
27/27 - 0s - loss: 0.1278 - accuracy: 0.9778 - 36ms/epoch - 1ms/step
Epoch 105/200
27/27 - 0s - loss: 0.1288 - accuracy: 0.9778 - 35ms/epoch - 1ms/step
Epoch 106/200
27/27 - 0s - loss: 0.1298 - accuracy: 0.9852 - 33ms/epoch - 1ms/step
Epoch 107/200
27/27 - 0s - loss: 0.1322 - accuracy: 0.9778 - 32ms/epoch - 1ms/step
Epoch 108/200
27/27 - 0s - loss: 0.1272 - accuracy: 0.9778 - 32ms/epoch - 1ms/step
Epoch 109/200
27/27 - 0s - loss: 0.1252 - accuracy: 0.9852 - 32ms/epoch - 1ms/step
Epoch 110/200
27/27 - 0s - loss: 0.1215 - accuracy: 0.9852 - 31ms/epoch - 1ms/step
Epoch 111/200
27/27 - 0s - loss: 0.1195 - accuracy: 0.9778 - 33ms/epoch - 1ms/step
Epoch 112/200
27/27 - 0s - loss: 0.1177 - accuracy: 0.9778 - 31ms/epoch - 1ms/step
Epoc

Epoch 200/200
27/27 - 0s - loss: 0.0858 - accuracy: 0.9704 - 34ms/epoch - 1ms/step
3/3 - 0s - loss: 0.0931 - accuracy: 1.0000 - 181ms/epoch - 60ms/step
Epoch 1/200
27/27 - 0s - loss: 1.0990 - accuracy: 0.2815 - 451ms/epoch - 17ms/step
Epoch 2/200
27/27 - 0s - loss: 1.0987 - accuracy: 0.3407 - 32ms/epoch - 1ms/step
Epoch 3/200
27/27 - 0s - loss: 1.0986 - accuracy: 0.3407 - 34ms/epoch - 1ms/step
Epoch 4/200
27/27 - 0s - loss: 1.0988 - accuracy: 0.3259 - 36ms/epoch - 1ms/step
Epoch 5/200
27/27 - 0s - loss: 1.0985 - accuracy: 0.3407 - 36ms/epoch - 1ms/step
Epoch 6/200
27/27 - 0s - loss: 1.0985 - accuracy: 0.3259 - 32ms/epoch - 1ms/step
Epoch 7/200
27/27 - 0s - loss: 1.0986 - accuracy: 0.3111 - 32ms/epoch - 1ms/step
Epoch 8/200
27/27 - 0s - loss: 1.0986 - accuracy: 0.3407 - 33ms/epoch - 1ms/step
Epoch 9/200
27/27 - 0s - loss: 1.0987 - accuracy: 0.2667 - 31ms/epoch - 1ms/step
Epoch 10/200
27/27 - 0s - loss: 1.0985 - accuracy: 0.3333 - 33ms/epoch - 1ms/step
Epoch 11/200
27/27 - 0s - loss: 1.0

27/27 - 0s - loss: 1.0982 - accuracy: 0.2741 - 33ms/epoch - 1ms/step
Epoch 100/200
27/27 - 0s - loss: 1.0983 - accuracy: 0.2889 - 36ms/epoch - 1ms/step
Epoch 101/200
27/27 - 0s - loss: 1.0982 - accuracy: 0.2963 - 41ms/epoch - 2ms/step
Epoch 102/200
27/27 - 0s - loss: 1.0982 - accuracy: 0.2667 - 35ms/epoch - 1ms/step
Epoch 103/200
27/27 - 0s - loss: 1.0983 - accuracy: 0.3407 - 33ms/epoch - 1ms/step
Epoch 104/200
27/27 - 0s - loss: 1.0982 - accuracy: 0.3185 - 34ms/epoch - 1ms/step
Epoch 105/200
27/27 - 0s - loss: 1.0983 - accuracy: 0.3407 - 33ms/epoch - 1ms/step
Epoch 106/200
27/27 - 0s - loss: 1.0983 - accuracy: 0.3407 - 33ms/epoch - 1ms/step
Epoch 107/200
27/27 - 0s - loss: 1.0984 - accuracy: 0.2815 - 35ms/epoch - 1ms/step
Epoch 108/200
27/27 - 0s - loss: 1.0983 - accuracy: 0.2963 - 33ms/epoch - 1ms/step
Epoch 109/200
27/27 - 0s - loss: 1.0983 - accuracy: 0.3259 - 33ms/epoch - 1ms/step
Epoch 110/200
27/27 - 0s - loss: 1.0982 - accuracy: 0.3185 - 35ms/epoch - 1ms/step
Epoch 111/200
27/2

Epoch 198/200
27/27 - 0s - loss: 1.0982 - accuracy: 0.3037 - 32ms/epoch - 1ms/step
Epoch 199/200
27/27 - 0s - loss: 1.0985 - accuracy: 0.3407 - 33ms/epoch - 1ms/step
Epoch 200/200
27/27 - 0s - loss: 1.0983 - accuracy: 0.3407 - 32ms/epoch - 1ms/step
3/3 - 0s - loss: 1.1082 - accuracy: 0.2667 - 179ms/epoch - 60ms/step
Epoch 1/200
27/27 - 1s - loss: 1.0979 - accuracy: 0.3556 - 686ms/epoch - 25ms/step
Epoch 2/200
27/27 - 0s - loss: 1.0626 - accuracy: 0.3556 - 39ms/epoch - 1ms/step
Epoch 3/200
27/27 - 0s - loss: 1.0417 - accuracy: 0.3556 - 32ms/epoch - 1ms/step
Epoch 4/200
27/27 - 0s - loss: 1.0294 - accuracy: 0.3556 - 33ms/epoch - 1ms/step
Epoch 5/200
27/27 - 0s - loss: 1.0183 - accuracy: 0.3556 - 32ms/epoch - 1ms/step
Epoch 6/200
27/27 - 0s - loss: 1.0068 - accuracy: 0.3556 - 32ms/epoch - 1ms/step
Epoch 7/200
27/27 - 0s - loss: 0.9935 - accuracy: 0.3556 - 32ms/epoch - 1ms/step
Epoch 8/200
27/27 - 0s - loss: 0.9819 - accuracy: 0.3556 - 34ms/epoch - 1ms/step
Epoch 9/200
27/27 - 0s - loss: 0

...

Epoch 186/200
27/27 - 0s - loss: 0.0840 - accuracy: 0.9704 - 34ms/epoch - 1ms/step
Epoch 187/200
27/27 - 0s - loss: 0.0828 - accuracy: 0.9778 - 32ms/epoch - 1ms/step
Epoch 188/200
27/27 - 0s - loss: 0.0820 - accuracy: 0.9704 - 33ms/epoch - 1ms/step
Epoch 189/200
27/27 - 0s - loss: 0.0853 - accuracy: 0.9704 - 34ms/epoch - 1ms/step
Epoch 190/200
27/27 - 0s - loss: 0.0796 - accuracy: 0.9778 - 34ms/epoch - 1ms/step
Epoch 191/200
27/27 - 0s - loss: 0.0794 - accuracy: 0.9630 - 34ms/epoch - 1ms/step
Epoch 192/200
27/27 - 0s - loss: 0.0799 - accuracy: 0.9704 - 32ms/epoch - 1ms/step
Epoch 193/200
27/27 - 0s - loss: 0.0809 - accuracy: 0.9778 - 33ms/epoch - 1ms/step
Epoch 194/200
27/27 - 0s - loss: 0.0810 - accuracy: 0.9704 - 33ms/epoch - 1ms/step
Epoch 195/200
27/27 - 0s - loss: 0.0764 - accuracy: 0.9704 - 32ms/epoch - 1ms/step
Epoch 196/200
27/27 - 0s - loss: 0.0776 - accuracy: 0.9704 - 34ms/epoch - 1ms/step
Epoch 197/200
27/27 - 0s - loss: 0.0789 - accuracy: 0.9704 - 33ms/epoch - 1ms/step
Epoc

Epoch 85/200
27/27 - 0s - loss: 0.1628 - accuracy: 0.9704 - 33ms/epoch - 1ms/step
Epoch 86/200
27/27 - 0s - loss: 0.1662 - accuracy: 0.9556 - 34ms/epoch - 1ms/step
Epoch 87/200
27/27 - 0s - loss: 0.1651 - accuracy: 0.9556 - 31ms/epoch - 1ms/step
Epoch 88/200
27/27 - 0s - loss: 0.1555 - accuracy: 0.9778 - 34ms/epoch - 1ms/step
Epoch 89/200
27/27 - 0s - loss: 0.1528 - accuracy: 0.9630 - 32ms/epoch - 1ms/step
Epoch 90/200
27/27 - 0s - loss: 0.1512 - accuracy: 0.9778 - 33ms/epoch - 1ms/step
Epoch 91/200
27/27 - 0s - loss: 0.1523 - accuracy: 0.9556 - 33ms/epoch - 1ms/step
Epoch 92/200
27/27 - 0s - loss: 0.1486 - accuracy: 0.9778 - 33ms/epoch - 1ms/step
Epoch 93/200
27/27 - 0s - loss: 0.1447 - accuracy: 0.9778 - 35ms/epoch - 1ms/step
Epoch 94/200
27/27 - 0s - loss: 0.1414 - accuracy: 0.9778 - 32ms/epoch - 1ms/step
Epoch 95/200
27/27 - 0s - loss: 0.1385 - accuracy: 0.9778 - 32ms/epoch - 1ms/step
Epoch 96/200
27/27 - 0s - loss: 0.1375 - accuracy: 0.9778 - 35ms/epoch - 1ms/step
Epoch 97/200
27/

Epoch 184/200
27/27 - 0s - loss: 0.0742 - accuracy: 0.9852 - 34ms/epoch - 1ms/step
Epoch 185/200
27/27 - 0s - loss: 0.0718 - accuracy: 0.9852 - 32ms/epoch - 1ms/step
Epoch 186/200
27/27 - 0s - loss: 0.0718 - accuracy: 0.9852 - 32ms/epoch - 1ms/step
Epoch 187/200
27/27 - 0s - loss: 0.0730 - accuracy: 0.9852 - 32ms/epoch - 1ms/step
Epoch 188/200
27/27 - 0s - loss: 0.0713 - accuracy: 0.9852 - 32ms/epoch - 1ms/step
Epoch 189/200
27/27 - 0s - loss: 0.0715 - accuracy: 0.9852 - 32ms/epoch - 1ms/step
Epoch 190/200
27/27 - 0s - loss: 0.0758 - accuracy: 0.9852 - 33ms/epoch - 1ms/step
Epoch 191/200
27/27 - 0s - loss: 0.0820 - accuracy: 0.9704 - 32ms/epoch - 1ms/step
Epoch 192/200
27/27 - 0s - loss: 0.0798 - accuracy: 0.9704 - 33ms/epoch - 1ms/step
Epoch 193/200
27/27 - 0s - loss: 0.0788 - accuracy: 0.9852 - 33ms/epoch - 1ms/step
Epoch 194/200
27/27 - 0s - loss: 0.0769 - accuracy: 0.9704 - 34ms/epoch - 1ms/step
Epoch 195/200
27/27 - 0s - loss: 0.0711 - accuracy: 0.9778 - 36ms/epoch - 1ms/step
Epoc

Epoch 83/200
27/27 - 0s - loss: 0.1812 - accuracy: 0.9704 - 32ms/epoch - 1ms/step
Epoch 84/200
27/27 - 0s - loss: 0.1804 - accuracy: 0.9704 - 32ms/epoch - 1ms/step
Epoch 85/200
27/27 - 0s - loss: 0.1780 - accuracy: 0.9704 - 33ms/epoch - 1ms/step
Epoch 86/200
27/27 - 0s - loss: 0.1683 - accuracy: 0.9630 - 32ms/epoch - 1ms/step
Epoch 87/200
27/27 - 0s - loss: 0.1710 - accuracy: 0.9630 - 32ms/epoch - 1ms/step
Epoch 88/200
27/27 - 0s - loss: 0.1655 - accuracy: 0.9481 - 33ms/epoch - 1ms/step
Epoch 89/200
27/27 - 0s - loss: 0.1682 - accuracy: 0.9481 - 32ms/epoch - 1ms/step
Epoch 90/200
27/27 - 0s - loss: 0.1559 - accuracy: 0.9704 - 32ms/epoch - 1ms/step
Epoch 91/200
27/27 - 0s - loss: 0.1569 - accuracy: 0.9778 - 35ms/epoch - 1ms/step
Epoch 92/200
27/27 - 0s - loss: 0.1529 - accuracy: 0.9704 - 33ms/epoch - 1ms/step
Epoch 93/200
27/27 - 0s - loss: 0.1516 - accuracy: 0.9704 - 32ms/epoch - 1ms/step
Epoch 94/200
27/27 - 0s - loss: 0.1496 - accuracy: 0.9630 - 34ms/epoch - 1ms/step
Epoch 95/200
27/

Epoch 182/200
27/27 - 0s - loss: 0.0791 - accuracy: 0.9704 - 33ms/epoch - 1ms/step
Epoch 183/200
27/27 - 0s - loss: 0.0797 - accuracy: 0.9704 - 33ms/epoch - 1ms/step
Epoch 184/200
27/27 - 0s - loss: 0.0783 - accuracy: 0.9630 - 34ms/epoch - 1ms/step
Epoch 185/200
27/27 - 0s - loss: 0.0771 - accuracy: 0.9778 - 35ms/epoch - 1ms/step
Epoch 186/200
27/27 - 0s - loss: 0.0797 - accuracy: 0.9852 - 36ms/epoch - 1ms/step
Epoch 187/200
27/27 - 0s - loss: 0.0860 - accuracy: 0.9630 - 39ms/epoch - 1ms/step
Epoch 188/200
27/27 - 0s - loss: 0.0781 - accuracy: 0.9778 - 34ms/epoch - 1ms/step
Epoch 189/200
27/27 - 0s - loss: 0.0792 - accuracy: 0.9704 - 33ms/epoch - 1ms/step
Epoch 190/200
27/27 - 0s - loss: 0.0814 - accuracy: 0.9556 - 33ms/epoch - 1ms/step
Epoch 191/200
27/27 - 0s - loss: 0.0756 - accuracy: 0.9704 - 33ms/epoch - 1ms/step
Epoch 192/200
27/27 - 0s - loss: 0.0762 - accuracy: 0.9704 - 33ms/epoch - 1ms/step
Epoch 193/200
27/27 - 0s - loss: 0.0744 - accuracy: 0.9704 - 33ms/epoch - 1ms/step
Epoc